In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pickle
%pylab inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE
from sklearn.externals import joblib
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
from nltk.corpus import stopwords
from collections import defaultdict, Counter

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [8]:
def build_nmf_model(df):

    my_stops = stopwords.words()
    leg_df = pd.read_csv("data/legislators-current.csv")
    last_names = [name.lower() for name in list(leg_df.last_name)]
    my_stops += last_names
    excess_words = ["statement", "congress", "senator", "senate", "congressman", "committee", "whether", "role","since",
                   "united", "states", "well", "much", "department", "congress", "like", "senators", "two", "three",
                   "mr", "ms", "chairman", "ranking", "member", "republicans", "democrats", "republican", "democratic",
                   "democrat", "put", "going", "said", "needs" ,"see", "get", "back", "going", "way", "last", "could"]
    my_stops += excess_words
    all_releases = list(df["Release"])
    corpus = list()
    lengths = defaultdict(int)
    for index, release in enumerate(all_releases):
        sents = nltk.sent_tokenize(release)
        lengths[index] = len(sents)
        corpus += sents

    tfidf = TfidfVectorizer(max_df=0.95, min_df=0.01, stop_words=my_stops,
                            token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1, 2))

    rel_tf = tfidf.fit_transform(corpus)
    
    nmf = NMF(n_components=100, init="random", random_state=42)

    rel_nmf = nmf.fit_transform(rel_tf)

    return rel_nmf

In [5]:
all_df = pd.read_pickle("/Users/danielgilberg/data_science/metis/nlp_partisanship/data/all_committees.pkl")
all_df
recent_df = all_df[all_df.Year > 2011]
recent_df["Release"] = recent_df.apply(lambda row: row["Release"].strip(), axis=1)
recent_df = recent_df[recent_df.Release != '']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
